# GOA-ON Inventory Summary

by Landung Setiawan and Emilio Mayorga

This ipython notebook goes through the GOA-ON Inventory in Google Sheets and extract information attributes that in principle should be based on a vocabulary, or should have limited, repeated terms/strings:
`source_doc`, `platform_type`, `type`, `organization`, `country`, and `parameters`. 

Updates:
- 7/10/2017. Initial counts for Parameter, Platform Type, Source Doc, Type, Organization, and Country. [`lsetiawan`](https://github.com/lsetiawan)

---

## Reading Google Sheet into Pandas Data Frame

In [1]:
# Import Necessary Libraries
%matplotlib inline
import os

import numpy as np
import pandas as pd

import eiutil.googlesheets as gsheet
import vizer.tsharvest.util as vhutil

In [2]:
vizername = 'goaon'
bOpenDB = False

In [3]:
vizer = vhutil.Vizer(vizername, bOpenDB)

In [4]:
# Initialize gpspread authorization
gc = gsheet.gspread_authorize(vizer)

In [5]:
# Names of Google Sheet GOA-ON documents to be used
inventory_master_gdocname = 'GOA-ON_structering'
inventory_mappings_gdocname = 'GOAONinventory_mappings'

In [6]:
# Open master inventory worksheet
df_xls = gsheet.get_googlesheet_df(gc, inventory_master_gdocname, 'inventory',
                                   data_rowstart=2)

In [7]:
df_xls.head()

,id,overlaps_with,comments_about_overlaps,source_doc,platform_type,platform_name,type,organization,organization_abbreviation,department,...,sensors,parameters,parameters_planned,method,depth_range,comments,longitude,latitude,location,method_documentation
0,2,,,FixedTimeSeries,FOTS,AK,IOLR Coastal Beach Rock Monitoring,National Institute of Oceangraphical and Limno...,NIOLR,,...,,dissolved oxygen; temperature; salinity; nutri...,dissolved inorganic carbon,,,,35.103906,33.063656,,
1,3,,,FixedTimeSeries,FOTS,"Alegria, CA",Intertidal (surf-zone) station,"University of California, Santa Barbara",UC Santa Barbara,,...,GIFET (pH),pH; pCO2,,discrete bottle samples,,"For a full listing of parameters, contact kris...",-120.28,34.47,,
2,4,,,FixedTimeSeries,FOTS,Anacapa Island,Pier-mounted sensor,"University of California, Santa Barbara",UC Santa Barbara,,...,SeaFET (pH),pH; temperature,,pH-sensor mounted to the pier piling at Anacap...,,,-119.3622111,34.01653611,,
3,6,,,FixedTimeSeries,FOTS,BHS UFS EMS,Unmanned Fireship,,,,...,Sea &amp; sun CTD 48; APP Automatic Pump Photo...,pH; pCO2; salinity; temperature; dissolved oxy...,,,3 - 30 m,,6.35,54.16667,,
4,7,,,FixedTimeSeries,FOTS,BHS UFS German Bight,Unmanned Fireship,,,,...,Sea &amp; sun CTD 48; APP Automatic Pump Photo...,pH; pCO2; salinity; temperature; dissolved oxy...,,,3 - 30 m,,7.45,54.16667,,


---
## Getting Parameter Counts

In [8]:
params_rec = list()
for i, df in df_xls.iterrows():
    parameters_str = df['parameters']
    param_lst = [token.strip() for token in parameters_str.split(';')]
    np_params = np.array(param_lst)
    unique, counts = np.unique(np_params, return_counts=True)
    
    params_rec.append(dict(zip(unique, counts)))

df_params = pd.DataFrame.from_records(params_rec)

**Note: There are some blanks, and strings not formatted correctly**

In [9]:
list(df_params.columns)

['',
 '13-C',
 '14-C',
 '18O-H2O',
 'Alkalinity',
 'Beam Att.',
 'C',
 'C flux',
 'CDOM',
 'CDOM fluorescence',
 'CFC',
 'CH4',
 'CO2_Water',
 'CO2_air',
 'CO2_sw',
 'Chlorophyll',
 'Chlorphyll',
 'DIC',
 'DIC-d13C',
 'DMS',
 'DOC',
 'DOM',
 'Disco Secchi',
 'Dissolved O2',
 'Dissolved oxygen',
 'Fe',
 'Fluorescence',
 'H2SO4',
 'HPLC',
 'Kd',
 'N flux',
 'N2O',
 'NO3',
 'Nutrients',
 'P flux',
 'PIC',
 'PO4',
 'POC',
 'SF6',
 'Salinity',
 'Sechi',
 'Si flux',
 'Sigma-t',
 'TCO2',
 'TDN',
 'TN',
 'Temperature',
 'Total Alkalinity (TA)',
 'Turbidity',
 'air pCO2',
 'air temperature',
 'alkalinity',
 'ammonium',
 'atmospheric pressure',
 'backscattering',
 'bacteria',
 'bacterial abundance',
 'bacterial production',
 'bacterial respiration rate',
 'biogenic particulate Si',
 'chlorophyll',
 'chlorophyll/pigments',
 'chloropigment',
 'cholorphyll/pigments',
 'conductivity',
 'copepod egg production',
 'current',
 'current direction',
 'current speed',
 'cyanobacteria',
 'd15N',
 'd18O',
 

In [10]:
param_counts = df_params.count()

In [11]:
param_counts

                                       91
13-C                                    2
14-C                                    1
18O-H2O                                 1
Alkalinity                              1
Beam Att.                              31
C                                       2
C flux                                 24
CDOM                                    2
CDOM fluorescence                       1
CFC                                    29
CH4                                     2
CO2_Water                               1
CO2_air                                36
CO2_sw                                 40
Chlorophyll                             1
Chlorphyll                              1
DIC                                     1
DIC-d13C                                9
DMS                                     3
DOC                                     1
DOM                                     1
Disco Secchi                            1
Dissolved O2                      

---
## Get Platform Type Counts

In [12]:
df_xls['platform_type'].value_counts(dropna=False)

STS       242
M         126
FOTS       97
TS         50
VOS        30
RH         27
OP          1
M; STS      1
Name: platform_type, dtype: int64

---
## Get Source Doc Counts with Platform Type

In [13]:
df_cleaned_pt = df_xls.fillna({'Platform_type': 'NA'})

In [14]:
df_cleaned_pt.groupby(['source_doc', 'platform_type']).size()

source_doc               platform_type
Bjoern_Times_Series_OCB  M                  3
                         STS              193
                         TS                48
Email_request            FOTS               1
                         M                  7
Fixed Ocean Time Series  FOTS               2
FixedTimeSeries          FOTS              78
                         OP                 1
                         STS                9
                         TS                 1
Mooring                  FOTS               2
                         M                103
                         M; STS             1
                         TS                 1
OOI                      M                 13
RepeatHydrography        RH                27
ShipbasedTime_Series     STS               39
VOS                      VOS               29
WESTPAC                  FOTS              11
on-line request          FOTS               3
                         STS             

---
## Get Type Counts

In [15]:
df_xls['type'].value_counts(dropna=False)

                                                                                        371
NOAA OA Coral Reef Monitoring Site                                                       20
Intertidal (surf-zone) station                                                           15
OA/pCO2 Mooring (Coastal)                                                                14
ChloroGIN - ANTARES Station                                                              11
OA/pCO2 Mooring (Open Ocean)                                                              9
Open Ocean Mooring                                                                        8
Inner-shelf mooring (15-20 meter depth)                                                   8
OA/pCO2 Mooring (Coral Reef)                                                              7
IOLR Hydrographic cruise station                                                          6
Ship-based Time Series                                                          

---
## Get Organization Counts

In [16]:
df_cleaned_org = df_xls.fillna({'organization': 'NA'})

**Note: There are some Repeating Organization or Similar, but spelled differently.**

In [17]:
df_cleaned_org.groupby(['organization', 'country']).size()

organization                                                                          country      
                                                                                                       22
                                                                                      Argentina         5
                                                                                      Australia         7
                                                                                      Barbados          1
                                                                                      Belgium           1
                                                                                      Brazil            2
                                                                                      Canada           46
                                                                                      Chile             4
                                                    

---
## Get Country Counts

In [18]:
country_counts = df_xls['country'].value_counts(dropna=False)

In [19]:
country_counts

US                        157
Canada                     61
                           49
France                     31
UK                         31
Mexico                     24
Australia                  21
Spain                      20
Norway                     20
Chile                      13
Israel                     12
Japan                      11
Germany                    11
Italy                       8
Brazil                      6
China                       6
South Africa                6
Ireland                     5
Argentina                   5
Korea                       5
India                       4
Peru                        4
Iceland                     4
Colombia                    4
Finland                     4
Sweden                      4
Denmark                     4
Thailand                    3
New Zealand                 3
Venezuela                   3
Malaysia                    2
Viet Nam                    2
Croatia                     2
Latvia    

---